<a href="https://colab.research.google.com/github/Aakankshavr/AI-lab/blob/main/1BM22CS001_Week7_FOL_Unification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def unify(term1, term2, substitution_set=None):
    """
    Unifies two terms using the unification algorithm.

    Args:
        term1: The first term (can be a variable, constant, or a function with arguments).
        term2: The second term.
        substitution_set: The current substitution set (default: empty).

    Returns:
        A substitution set if the terms are unifiable, otherwise None.
    """
    if substitution_set is None:
        substitution_set = {}


    term1 = apply_substitution_to_term(term1, substitution_set)
    term2 = apply_substitution_to_term(term2, substitution_set)


    if term1 == term2:
        return substitution_set


    if isinstance(term1, str) and term1.isupper():
        return unify_variable(term1, term2, substitution_set)
    if isinstance(term2, str) and term2.isupper():
        return unify_variable(term2, term1, substitution_set)


    if isinstance(term1, list) and isinstance(term2, list):
        if term1[0] != term2[0] or len(term1) != len(term2):
            return None
        for arg1, arg2 in zip(term1[1:], term2[1:]):
            substitution_set = unify(arg1, arg2, substitution_set)
            if substitution_set is None:
                return None
        return substitution_set


    return None

def unify_variable(var, term, substitution_set):
    """
    Unifies a variable with another term.

    Args:
        var: The variable to unify.
        term: The term to unify with.
        substitution_set: The current substitution set.

    Returns:
        An updated substitution set if unification is successful, otherwise None.
    """
    if var in substitution_set:
        return unify(substitution_set[var], term, substitution_set)
    if term in substitution_set:
        return unify(var, substitution_set[term], substitution_set)
    if occurs_check(var, term):
        return None
    substitution_set[var] = term
    return substitution_set

def occurs_check(var, term):
    """
    Checks if a variable occurs in a term (to prevent cyclic substitutions).

    Args:
        var: The variable to check.
        term: The term to check against.

    Returns:
        True if the variable occurs in the term, otherwise False.
    """
    if var == term:
        return True
    if isinstance(term, list):
        return any(occurs_check(var, arg) for arg in term[1:])
    return False

def apply_substitution_to_term(term, substitution_set):
    """
    Applies a substitution set to a term.

    Args:
        term: The term to which the substitution is applied.
        substitution_set: The substitution set.

    Returns:
        The term after applying substitutions.
    """
    if isinstance(term, str):
        return substitution_set.get(term, term)
    return [term[0]] + [apply_substitution_to_term(arg, substitution_set) for arg in term[1:]]

def parse_term(term_str):
    """
    Parses a term string into a structured format.

    Args:
        term_str: The string representation of the term.

    Returns:
        A structured term (list).
    """
    term_str = term_str.strip()
    if '(' not in term_str:
        return term_str

    func_name, args_str = term_str.split('(', 1)
    args_str = args_str.rstrip(')')

    args = []
    bracket_count = 0
    current_arg = []

    for char in args_str:
        if char == ',' and bracket_count == 0:
            args.append(''.join(current_arg).strip())
            current_arg = []
        else:
            if char == '(':
                bracket_count += 1
            elif char == ')':
                bracket_count -= 1
            current_arg.append(char)

    if current_arg:
        args.append(''.join(current_arg).strip())

    return [func_name] + [parse_term(arg) for arg in args]

def main():
    term1_str = input("Enter the first term: ")
    term2_str = input("Enter the second term: ")

    term1 = parse_term(term1_str)
    term2 = parse_term(term2_str)

    substitution_set = unify(term1, term2)

    if substitution_set is not None:
        unified_term1 = apply_substitution_to_term(term1, substitution_set)
        unified_term2 = apply_substitution_to_term(term2, substitution_set)
        print("Unified Term 1:", unified_term1)
        print("Unified Term 2:", unified_term2)
        print("Substitution Set:", substitution_set)
    else:
        print("The terms are not unifiable.")

if __name__ == "__main__":
    main()


Enter the first term: f(X,a,Y)
Enter the second term: f(Z,a,b)
Unified Term 1: ['f', 'Z', 'a', 'b']
Unified Term 2: ['f', 'Z', 'a', 'b']
Substitution Set: {'X': 'Z', 'Y': 'b'}
